In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

/Users/houhiroshisakai/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/houhiroshisakai/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Load the Stanford Dogs dataset
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'stanford_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)


In [14]:
# Function to resize and normalize images
def preprocess(image, label):
    image = tf.cast(image, tf.float32)
    image = (image/255.0)  # Normalize pixel values
    image = tf.image.resize(image, (224, 224))  # Resize images
    return image, label


In [4]:
# Apply preprocessing to dataset
train = raw_train.map(preprocess)
validation = raw_validation.map(preprocess)
test = raw_test.map(preprocess)

In [5]:
# Batch and prefetch
BATCH_SIZE = 32
train_batches = train.shuffle(1000).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation.batch(BATCH_SIZE).prefetch(1)
test_batches = test.batch(BATCH_SIZE).prefetch(1)

In [15]:
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.5)
    return image, label

# Apply the `augment` function to each item in the training set using the map method
train_batches = train_batches.map(augment)

In [6]:
# Model Building
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Number of classes in the dataset
num_classes = metadata.features['label'].num_classes

# Building the CNN model
model = Sequential([
    # Convolutional Layer 1
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),

    # Convolutional Layer 2
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    # Convolutional Layer 3
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    # Flatten the results to feed into a Dense layer
    Flatten(),
    Dropout(0.5),

    # 512 neuron hidden layer
    Dense(512, activation='relu'),
    Dense(num_classes, activation='softmax')  # Output layer
])

# Model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [9]:
# Model Training and Optimization
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_batches,
    validation_data=validation_batches,
    epochs=20,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss')]
)

Epoch 1/20
300/300 [==============================] - 104s 344ms/step - loss: 4.4639 - accuracy: 0.0437 - val_loss: 4.5507 - val_accuracy: 0.0317
Epoch 2/20
300/300 [==============================] - 103s 342ms/step - loss: 3.6465 - accuracy: 0.1809 - val_loss: 5.2829 - val_accuracy: 0.0383
Epoch 3/20
300/300 [==============================] - 103s 342ms/step - loss: 1.7831 - accuracy: 0.5653 - val_loss: 7.5033 - val_accuracy: 0.0383
Epoch 4/20
300/300 [==============================] - 103s 342ms/step - loss: 0.7723 - accuracy: 0.8116 - val_loss: 9.2792 - val_accuracy: 0.0258


In [10]:
# Evaluating the model
test_loss, test_accuracy = model.evaluate(test_batches)
print(f"Test accuracy: {test_accuracy}")

# Confusion Matrix
y_pred = model.predict(test_batches)
y_true = tf.concat([y for x, y in test_batches], axis=0)
confusion_mtx = tf.math.confusion_matrix(y_true, tf.argmax(y_pred, axis=1))

38/38 [==============================] - 4s 100ms/step - loss: 9.4205 - accuracy: 0.0333
Test accuracy: 0.03333333507180214
38/38 [==============================] - 4s 95ms/step


In [11]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
    # Add more transformations as needed
])

# Apply data augmentation to the training dataset
train_batches = train_batches.map(lambda x, y: (data_augmentation(x, training=True), y))

In [12]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2

# Adjust the model architecture
model = Sequential([
    # Add data augmentation as the first layer
    data_augmentation,

    # Convolutional Layer 1
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3), kernel_regularizer=l2(0.001)),
    MaxPooling2D(2, 2),

    # Add additional layers with regularization and dropout as needed

    # Output layer with L2 Regularization
    Dense(num_classes, activation='softmax', kernel_regularizer=l2(0.001))
])

# Compile the model again
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
# EarlyStopping callback to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True)

# Train the model
history = model.fit(
    train_batches,
    validation_data=validation_batches,
    epochs=50,  # Increase if necessary
    callbacks=[early_stopping]
)

Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/runpy.py", line 87, in _run_code

  File "/Users/houhiroshisakai/Library/Python/3.9/lib/python/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/houhiroshisakai/Library/Python/3.9/lib/python/site-packages/traitlets/config/application.py", line 1043, in launch_instance

  File "/Users/houhiroshisakai/Library/Python/3.9/lib/python/site-packages/ipykernel/kernelapp.py", line 725, in start

  File "/Users/houhiroshisakai/Library/Python/3.9/lib/python/site-packages/tornado/platform/asyncio.py", line 195, in sta